In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import spearmanr
import os
from datetime import datetime

def calculate_spearman_correlation_with_groups(file_path, predicted_col, true_col, output_dir):
    """
    CSVファイルから相関係数を計算し、範囲別に分析する関数
    
    Parameters:
    file_path (str): 入力CSVファイルのパス
    predicted_col (str): 予測値の列名
    true_col (str): 実測値の列名
    output_dir (str): 出力ディレクトリのパス
    
    Returns:
    dict: 計算結果の辞書
    """
    
    try:
        # CSVファイルの読み込み
        print(f"ファイルを読み込み中: {file_path}")
        df = pd.read_csv(file_path, encoding='utf-8')
        print(f"データ読み込み完了: {len(df)}行 x {len(df.columns)}列")
        
        # データの最初の数行を表示
        print(f"\nデータの最初の5行:")
        print(df.head())
        
        # 列名の確認
        print(f"利用可能な列名: {list(df.columns)}")
        
        # 指定された列が存在するかチェック
        if predicted_col not in df.columns:
            raise ValueError(f"予測値の列 '{predicted_col}' が見つかりません")
        if true_col not in df.columns:
            raise ValueError(f"実測値の列 '{true_col}' が見つかりません")
        
        # 欠損値の処理
        df_clean = df[[predicted_col, true_col]].dropna()
        print(f"欠損値除去後のデータ数: {len(df_clean)}行")
        
        if len(df_clean) < 3:
            raise ValueError("有効なデータが3件未満です。相関係数の計算ができません。")
        
        # 予測値と実測値を取得
        predicted_values = df_clean[predicted_col].values
        true_values = df_clean[true_col].values
        
        # 実測値の統計情報を表示
        print(f"\n実測値の統計情報:")
        print(f"最小値: {true_values.min()}")
        print(f"最大値: {true_values.max()}")
        print(f"平均値: {true_values.mean():.2f}")
        print(f"標準偏差: {true_values.std():.2f}")
        
        print(f"\n予測値の統計情報:")
        print(f"最小値: {predicted_values.min():.2f}")
        print(f"最大値: {predicted_values.max():.2f}")
        print(f"平均値: {predicted_values.mean():.2f}")
        print(f"標準偏差: {predicted_values.std():.2f}")
        
        # 各グループのデータ数を事前確認
        group1_count = len(df_clean[df_clean[true_col] <= 19])
        group2_count = len(df_clean[(df_clean[true_col] >= 20) & (df_clean[true_col] <= 25)])
        group3_count = len(df_clean[df_clean[true_col] >= 26])
        
        print(f"\nグループ別データ数:")
        print(f"19点以下: {group1_count}件")
        print(f"20-25点: {group2_count}件")
        print(f"26点以上: {group3_count}件")
        
        # 全体のスピアマン相関係数を計算
        overall_corr, overall_p_value = spearmanr(predicted_values, true_values)
        
        # 結果格納用の辞書
        results = {
            'overall': {
                'data_count': len(df_clean),
                'correlation': overall_corr,
                'p_value': overall_p_value
            }
        }
        
        # 実測値の範囲別に分析
        # 19点以下
        group1 = df_clean[df_clean[true_col] <= 19]
        if len(group1) >= 3:
            corr1, p1 = spearmanr(group1[predicted_col], group1[true_col])
            results['group_19_below'] = {
                'data_count': len(group1),
                'correlation': corr1,
                'p_value': p1,
                'true_mean': group1[true_col].mean(),
                'predicted_mean': group1[predicted_col].mean()
            }
        else:
            results['group_19_below'] = {
                'data_count': len(group1),
                'correlation': np.nan,
                'p_value': np.nan,
                'true_mean': group1[true_col].mean() if len(group1) > 0 else np.nan,
                'predicted_mean': group1[predicted_col].mean() if len(group1) > 0 else np.nan,
                'note': 'データ数不足（3件未満）'
            }
        
        # 20-25点
        group2 = df_clean[(df_clean[true_col] >= 20) & (df_clean[true_col] <= 25)]
        if len(group2) >= 3:
            corr2, p2 = spearmanr(group2[predicted_col], group2[true_col])
            results['group_20_25'] = {
                'data_count': len(group2),
                'correlation': corr2,
                'p_value': p2,
                'true_mean': group2[true_col].mean(),
                'predicted_mean': group2[predicted_col].mean()
            }
        else:
            results['group_20_25'] = {
                'data_count': len(group2),
                'correlation': np.nan,
                'p_value': np.nan,
                'true_mean': group2[true_col].mean() if len(group2) > 0 else np.nan,
                'predicted_mean': group2[predicted_col].mean() if len(group2) > 0 else np.nan,
                'note': 'データ数不足（3件未満）' if len(group2) < 3 else 'データなし'
            }
        
        # 26点以上
        group3 = df_clean[df_clean[true_col] >= 26]
        if len(group3) >= 3:
            corr3, p3 = spearmanr(group3[predicted_col], group3[true_col])
            results['group_26_above'] = {
                'data_count': len(group3),
                'correlation': corr3,
                'p_value': p3,
                'true_mean': group3[true_col].mean(),
                'predicted_mean': group3[predicted_col].mean()
            }
        else:
            results['group_26_above'] = {
                'data_count': len(group3),
                'correlation': np.nan,
                'p_value': np.nan,
                'true_mean': group3[true_col].mean() if len(group3) > 0 else np.nan,
                'predicted_mean': group3[predicted_col].mean() if len(group3) > 0 else np.nan,
                'note': 'データ数不足（3件未満）' if len(group3) < 3 else 'データなし'
            }
        
        # 結果の表示
        print("\n=== スピアマン相関係数の結果 ===")
        print(f"全体のデータ数: {results['overall']['data_count']}")
        print(f"全体の相関係数: {results['overall']['correlation']:.4f}")
        print(f"全体のp値: {results['overall']['p_value']:.4f}")
        
        print("\n=== 範囲別分析結果 ===")
        for group_name, group_data in results.items():
            if group_name == 'overall':
                continue
            
            group_label = {
                'group_19_below': '19点以下',
                'group_20_25': '20-25点',
                'group_26_above': '26点以上'
            }[group_name]
            
            print(f"\n{group_label}:")
            print(f"  データ数: {group_data['data_count']}")
            if not np.isnan(group_data['correlation']):
                print(f"  相関係数: {group_data['correlation']:.4f}")
                print(f"  p値: {group_data['p_value']:.4f}")
                print(f"  実測値平均: {group_data['true_mean']:.2f}")
                print(f"  予測値平均: {group_data['predicted_mean']:.2f}")
            else:
                print(f"  相関係数: 計算不可 ({group_data.get('note', 'データなし')})")
        
        # 結果をCSVファイルとして保存
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_file = os.path.join(output_dir, f"correlation_analysis_{timestamp}.csv")
        
        # 散布図データも保存
        scatter_file = os.path.join(output_dir, f"scatter_data_{timestamp}.csv")
        scatter_data = df_clean[[predicted_col, true_col]].copy()
        scatter_data['Group'] = scatter_data[true_col].apply(
            lambda x: '19点以下' if x <= 19 else ('20-25点' if 20 <= x <= 25 else '26点以上')
        )
        scatter_data.to_csv(scatter_file, index=False, encoding='utf-8-sig')
        print(f"散布図用データを保存しました: {scatter_file}")
        
        # 結果をDataFrameに変換
        output_data = []
        for group_name, group_data in results.items():
            group_label = {
                'overall': '全体',
                'group_19_below': '19点以下',
                'group_20_25': '20-25点',
                'group_26_above': '26点以上'
            }.get(group_name, group_name)
            
            row = {
                'グループ': group_label,
                'データ数': group_data['data_count'],
                'スピアマン相関係数': group_data['correlation'],
                'p値': group_data['p_value']
            }
            
            if 'true_mean' in group_data:
                row['実測値平均'] = group_data['true_mean']
                row['予測値平均'] = group_data['predicted_mean']
            
            if 'note' in group_data:
                row['備考'] = group_data['note']
            
            output_data.append(row)
        
        output_df = pd.DataFrame(output_data)
        output_df.to_csv(output_file, index=False, encoding='utf-8-sig')
        print(f"\n結果を保存しました: {output_file}")
        
        return results
        
    except Exception as e:
        print(f"エラーが発生しました: {str(e)}")
        return None

def main():
    """
    メイン実行関数
    """
    # ファイルパスの設定
    file_path = r"G:\共有ドライブ\GAP_長寿研\user\iwamoto\視線の動きの俊敏さ\smoteresult\features_0509kyosoku_smote_density_n721_20250513_191000\result\prediction_results_20250518_164911.csv"
    # 出力ディレクトリの設定
    output_dir = r"G:\共有ドライブ\GAP_長寿研\user\iwamoto\視線の動きの俊敏さ\smoteresult\features_0509kyosoku_smote_density_n721_20250513_191000\result"
    
    # 列名の設定（実データに対応）
    predicted_col = 'Predicted_Value'  # 予測値の列名
    true_col = 'True_Value'            # 実測値の列名
    
    # 出力ディレクトリが存在しない場合は作成
    os.makedirs(output_dir, exist_ok=True)
    
    # 相関係数の計算と分析を実行
    results = calculate_spearman_correlation_with_groups(
        file_path=file_path,
        predicted_col=predicted_col,
        true_col=true_col,
        output_dir=output_dir
    )
    
    if results:
        print("\n分析が正常に完了しました。")
    else:
        print("\n分析中にエラーが発生しました。")

# 汎用関数：任意のファイルと列名で実行
def analyze_correlation(file_path, predicted_col, true_col, output_dir, 
                       group_thresholds=None):
    """
    汎用的な相関係数分析関数
    
    Parameters:
    file_path (str): 入力CSVファイルのパス
    predicted_col (str): 予測値の列名
    true_col (str): 実測値の列名
    output_dir (str): 出力ディレクトリのパス
    group_thresholds (dict): グループ分けの閾値 (デフォルト: {'low': 19, 'high': 26})
    
    Example:
    analyze_correlation(
        file_path='data.csv',
        predicted_col='predicted',
        true_col='actual',
        output_dir='output',
        group_thresholds={'low': 15, 'high': 30}
    )
    """
    if group_thresholds is None:
        group_thresholds = {'low': 19, 'high': 26}
    
    return calculate_spearman_correlation_with_groups(
        file_path, predicted_col, true_col, output_dir
    )

if __name__ == "__main__":
    main()

ファイルを読み込み中: G:\共有ドライブ\GAP_長寿研\user\iwamoto\視線の動きの俊敏さ\smoteresult\features_0509kyosoku_smote_density_n721_20250513_191000\predictions_catboostregressor_smote_density.csv
データ読み込み完了: 65行 x 5列

データの最初の5行:
    InspectionDateAndId  True_Value  Predicted_Value     Error  Abs_Error
0  20240229091403_31629          25        25.879927 -0.879927   0.879927
1  20240213115102_31197          20        23.584659 -3.584659   3.584659
2  20240301090054_31697          24        25.770760 -1.770760   1.770760
3  20240201122019_20882          25        22.835361  2.164639   2.164639
4  20240208110617_31127          23        24.872012 -1.872012   1.872012
利用可能な列名: ['InspectionDateAndId', 'True_Value', 'Predicted_Value', 'Error', 'Abs_Error']
欠損値除去後のデータ数: 65行

実測値の統計情報:
最小値: 15
最大値: 30
平均値: 24.12
標準偏差: 3.94

予測値の統計情報:
最小値: 17.01
最大値: 29.33
平均値: 23.89
標準偏差: 2.47

グループ別データ数:
19点以下: 12件
20-25点: 22件
26点以上: 31件

=== スピアマン相関係数の結果 ===
全体のデータ数: 65
全体の相関係数: 0.5912
全体のp値: 0.0000

=== 範囲別分析結果 ===

19点以下:
  データ数: 12
